In [ ]:
import fiona
import os
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
from ipywidgets import HBox, Label, IntSlider,IntRangeSlider
from ipywidgets import HTML
from IPython.display import clear_output
from ipyleaflet import Map, basemaps, basemap_to_tiles, CircleMarker, Popup, Polyline, Polygon, LayerGroup, LayersControl,ScaleControl, FullScreenControl
from ipyleaflet import WidgetControl
from ipywidgets import interactive
import matplotlib as mpl
import ipywidgets as widgets
from ipywidgets import HBox, Label, IntSlider,IntRangeSlider
#import matplotlib.patches as patches
#from matplotlib.patches import Polygon
import matplotlib
import matplotlib.colors
import contextily as cx
import shapely.geometry as sg
from rijksdriehoek import rijksdriehoek
from matplotlib.collections import PatchCollection
import ipympl
import warnings
import numpy as np
import ast
import datetime as dt


RD = rijksdriehoek.Rijksdriehoek()
plt.rcParams.update({'font.size': 24})
MAX_GROUP_SIZE = 250

In [ ]:
def read_gpkg(filename):
    
    assert '.' in filename, f'Filename {filename} does not have an extension!'    
    if filename.split('.')[1] != 'gpkg':
        raise ValueError(f'Expected gpkg, got {filename}')
    if not os.path.isfile(f'Data/{filename}'):
        raise ValueError(f'File {filename} not found in Data directory!')
    
    AOI = {}
    with fiona.open(f"Data/{filename}") as layer:
        for feature in layer:     
            AOI[feature['properties']['ogc_fid']] = {
                "crd_dec": [feature['geometry']['coordinates'][ii] for ii in range(len(feature['geometry']['coordinates']))]
                }
            for key in ['h0', 's_h0', 'xI', 's_xI', 'xP', 's_xP', 'xE', 's_xE', 'tau', 's_tau',
                        'rho_xIh0', 'rho_xPh0', 'rho_xEh0', 'rho_tauh0',
                        'rho_xPxI', 'rho_xExI', 'rho_xItau', 
                        'rho_xPxE', 'rho_xPtau', 
                        'rho_xEtau', 'xI_frac']:
                AOI[feature['properties']['ogc_fid']][key] = feature['properties'][key]
    return AOI

def read_visualisation_gpkg(filename):
    assert '.' in filename, f'Filename {filename} does not have an extension!'    
    if filename.split('.')[1] != 'gpkg':
        raise ValueError(f'Expected gpkg, got {filename}')
    if not os.path.isfile(f'Data/{filename}'):
        raise ValueError(f'File {filename} not found in Data directory!')
    
    AOI = {}
    with fiona.open(f"Data/{filename}") as layer:
        for feature in layer:     
            AOI[feature['properties']['ogc_fid']] = {
                "crd_dec": [feature['geometry']['coordinates'][ii] for ii in range(len(feature['geometry']['coordinates']))]
                }
            for key in ['h2015_m', 'sigma_h2015_m', #'h2023_m', 'v_mpy', 'sigma_v_mpy', 
                   'xI_mpy', 'sigma_xI_mpy', 'xI_frac_1000pct', 
                   'mean_yearly_irreversible_subsidence_MYIS_mpy', 'sigma_MYIS_mpy',
                   'xP_mpmm', 'sigma_xP_mpmm', 'xE_mpmm', 'sigma_xE_mpmm', 'tau_kd', 'sigma_tau_kd',
                   'rho_h2015_xI_1000', 'rho_h2015_xP_1000', 'rho_h2015_xE_1000', 'rho_h2015_tau_1000',
                   'rho_xI_xP_1000', 'rho_xI_xE_1000', 'rho_xI_tau_1000', 
                   'rho_xP_xE_1000', 'rho_xP_tau_1000', 
                   'rho_xE_tau_1000',
                   'timespan_ky', 'n_observations_1000', 
                   'significant_irreversible_subsidence_detected_2s_1000', 'significant_irreversible_subsidence_detected_3s_1000', 
                   'discriminatory_power_2s_1000pct', 
                   'discriminatory_power_3s_1000pct', 'MDD_80p_mpy', 
                   'cum_irreversible_subsidence_since_2000_m', 'oldest_observation_ky', 
                   'OMT_sust_2s_1000', 'OMT_sust_3s_1000']:
                AOI[feature['properties']['ogc_fid']][key] = feature['properties'][key]
    return AOI

def convert_wgs84_to_rd(list_of_coords):
    """
    Converts a list of coordinates in WGS84 to RD
    :param list_of_coords: list of (lon, lat) objects
    :return: list of (rd_x, rd_y) objects
    """
    rd_coords = []
    for coord in list_of_coords:
        RD.from_wgs(lon=coord[0], lat=coord[1])
        rd_coords.append([RD.rd_x, RD.rd_y])
    return rd_coords

def convert_rd_to_wgs84(list_of_coords):
    """
    Converts a list of coordinates in RD to WGS
    :param list_of_coords: list of (rdx, rdy) objects
    :return: list of (lon, lat) objects
    """
    wgs_coords = []
    for coord in list_of_coords:
        RD.rd_x = coord[0]
        RD.rd_y = coord[1]
        wgs_coords.append(RD.to_wgs())
    return wgs_coords

def get_yearfrac(date):
    year = eval(date[:4])
    month = eval(date[5:7].lstrip('0'))
    day = eval(date[8:].lstrip('0'))
    dayOfYear = dt.datetime(year=year, month=month, day=day) - dt.datetime(year=year, month=1, day=1)
    yearDays = dt.datetime(year=year+1, month=1, day=1) - dt.datetime(year=year, month=1, day=1)
    yearfrac = year + dayOfYear.days / yearDays.days
    return yearfrac

In [ ]:
def read_aoi():
    filename = "Data/AoI_GreenHeart-polygon.shp"
    shape = fiona.open(filename)
    finished = False
    shape_iter = iter(shape)
    aoi_rd_poly = None
    while not finished:
        try:
            shp = next(shape_iter)
            polygons = [sg.Polygon(convert_wgs84_to_rd(shp["geometry"]["coordinates"][ii])) for ii in range(len(shp["geometry"]["coordinates"]))]
            if len(polygons) > 1 or aoi_rd_poly is not None:
                raise ValueError("Cannot handle AOI with more than 1 polygon!")
            aoi_rd_poly = polygons[0]
            if not aoi_rd_poly.is_valid:
                raise ValueError("Invalid polygon AOI!")
        except StopIteration:
            finished = True
    if aoi_rd_poly is None:
        raise ValueError(f"No valid AOI polygon found in {filename}!")
    return convert_rd_to_wgs84(np.array(aoi_rd_poly.exterior.coords.xy).T)
aoi_wgs_poly = read_aoi()

In [ ]:
def read_KNMI_file(filename):
    f = open(filename)
    data = f.read().split('\n')
    f.close()
    start_idx = None
    for i in range(len(data)):
        if '# STN,YYYYMMDD' in data[i]:
            start_idx = i + 2
            break
    headers = data[start_idx-2].split(',')
    headers = [h.strip() for h in headers]
    evapo_id = headers.index('EV24')
    date_id = headers.index('YYYYMMDD')
    precip_id = headers.index('RH')
    temp_id = headers.index('TX')
    processed_data = {}
    for row in data[start_idx:]:
        if row != '':
            split_row = row.split(',')
            date = dt.datetime.strptime(split_row[date_id].strip(), '%Y%m%d')
            if '' not in [date, split_row[evapo_id].strip(), split_row[precip_id].strip(), split_row[temp_id].strip()]:
                if '348' in filename and date.year < 1988: # very incomplete data before 1988
                    continue
                processed_data[date] = [eval(split_row[precip_id].strip()),
                                        eval(split_row[evapo_id].strip()),
                                        eval(split_row[temp_id].strip())]
                if processed_data[date][0] == -1:
                    processed_data[date][0] = 0
                processed_data[date][0] /= 10
                processed_data[date][1] /= 10
                processed_data[date][2] /= 10
    return processed_data

In [ ]:
gpkg_data = read_gpkg('InSAR_ALS_GNSS_Grav_Leveling.gpkg')

In [ ]:
vis_data = read_visualisation_gpkg('Visualisation_export.gpkg')

In [ ]:
t0 = '2015-01-02'
Cabauw = read_KNMI_file(f'Data/etmgeg_348.txt')
tmax = max(Cabauw.keys())
tmax = '{:0>4d}-{:0>2d}-{:0>2d}'.format(tmax.year, tmax.month, tmax.day)
tmin = min(Cabauw.keys()) + dt.timedelta(days=365) # to give space for tau to run back
tmin = '{:0>4d}-{:0>2d}-{:0>2d}'.format(tmin.year, tmin.month, tmin.day)
etmgeg = Cabauw 

tmin_yf = get_yearfrac(tmin)
tmax_yf = get_yearfrac(tmax)
t0_yf = get_yearfrac(t0)
t0_dt = dt.datetime.strptime(t0, '%Y-%m-%d')
tmin_dt = dt.datetime.strptime(tmin, '%Y-%m-%d')
tmax_dt = dt.datetime.strptime(tmax, '%Y-%m-%d')

t_eval_min = dt.datetime(1960, 1, 1)

In [ ]:
def fit_SPAMS_extended(EUP, t0, tmin, tmax, force_noInSAR=False):
    # print(EUP)
    InSAR = False
    if EUP['InSAR'] != -999 and not force_noInSAR:
        InSAR = True
        
    def no_insar_model():
        t0_yf = get_yearfrac(t0)
        t_data = []
        s_data = []
        h_data = []
        if EUP['EUP_type'] == 'brp':
            for idx in ['ThMD', 'AHN1', 'AHN2', 'AHN3', 'AHN4']:
                if idx != 'ThMD':
                    if -999 not in [EUP[f'{q}100_{idx}'] for q in 'hst'] and '' not in [EUP[f'{q}100_{idx}'] for q in 'hst']:
                        h_data.append(EUP[f'h100_{idx}'])
                        s_data.append(EUP[f's100_{idx}'])
                        t_data.append(get_yearfrac(EUP[f't100_{idx}']))
                else:
                    if -999 not in [EUP[f'{q}_{idx}'] for q in 'hst'] and '' not in [EUP[f'{q}_{idx}'] for q in 'hst']:
                        h_data.append(EUP[f'h_{idx}'])
                        s_data.append(EUP[f's_{idx}'])
                        t_data.append(get_yearfrac(EUP[f't_{idx}']))
                                                    
        else:
            for idx in ['ThMD', 'AHN1', 'AHN2', 'AHN3', 'AHN4']:
                if idx != 'ThMD':
                    if -999 not in [EUP[f'{q}100_{idx}'] for q in 'hst'] and '' not in [EUP[f'{q}100_{idx}'] for q in 'hst']:
                        h_data.append(EUP[f'h100_{idx}'])
                        s_data.append(EUP[f's100_{idx}'])
                        t_data.append(get_yearfrac(EUP[f't100_{idx}']))
                else:
                    if -999 not in [EUP[f'{q}_{idx}'] for q in 'hst'] and '' not in [EUP[f'{q}_{idx}'] for q in 'hst']:
                        h_data.append(EUP[f'h_{idx}'])
                        s_data.append(EUP[f's_{idx}'])
                        t_data.append(get_yearfrac(EUP[f't_{idx}']))
                        
        if len(h_data) < 2:
            return ('', (-999, -999), 0, [[-999, -999], [-999, -999]])
        
        time_basis = max(t_data) - min(t_data)
        
        y = np.zeros((len(t_data), 1))
        A = np.ones((len(t_data), 2))
        W = np.zeros((len(t_data), len(t_data)))
        
        for i in range(len(t_data)):
            y[i, 0] = h_data[i]
            A[i, 0] = (t_data[i] - t0_yf)/1000
            if s_data[i] > 1e-8:
                W[i, i] = 1/s_data[i]**2
            else:
                s = max(s_data)
                if s > 1e-8:
                    W[i, i] = 1/s**2
                else:
                    W[i, i] = 1 
                    
        Pa = np.linalg.inv(A.T @ W @ A) @ A.T @ W
        sol = Pa @ y
        Qy = Pa @ np.linalg.inv(W) @ Pa.T
        
        EoM_est = f"h={round(sol[0][0], 4)}(t-t0)/1000+{round(sol[1][0], 4)}"
        vel_uncertainty = np.sqrt(Qy[0,0])
        h_uncertainty = np.sqrt(Qy[1,1])
    
        return (EoM_est, (vel_uncertainty, h_uncertainty), time_basis, Qy)
    
    if InSAR: # has to be brp
        initial_params = None
        if 'xI' in EUP.keys() and 'xE' in EUP.keys() and 'xP' in EUP.keys() and 'tau' in EUP.keys() and 'h0' in EUP.keys():
            if EUP['xI'] not in [None, -999]:
                if EUP['xE'] not in [None, -999]:
                    if EUP['xP'] not in [None, -999]:
                        if EUP['h0'] not in [None, -999]:
                            if EUP['tau'] not in [None, -999]:
                                initial_params = [EUP['xP'], EUP['xE'], EUP['xI'], EUP['tau'], EUP['h0']]
        if initial_params is None:
            for idx in [37, 88, 110, 161]:
                if f'p_{idx}' in EUP['InSAR'].keys():
                    initial_params = list(EUP['InSAR'][f'p_{idx}'])
                    initial_params.append(EUP['h100_AHN3']) # closest to t0, ~2015
                    break

            if initial_params is None:
                initial_params = [np.nan] * 5
        if None in initial_params:
            for idx in [37, 88, 110, 161]:
                if f'p_{idx}' in EUP['InSAR'].keys():
                    initial_params = list(EUP['InSAR'][f'p_{idx}'])
                    initial_params.append(EUP['h100_AHN3']) # closest to t0, ~2015
                    break

            if initial_params is None:
                initial_params = [np.nan] * 5            
        #tau = initial_params[3]
        #_ = initial_params.pop(3)
        
        tmin_yf = get_yearfrac(tmin)
        tmax_yf = get_yearfrac(tmax)
        t0_yf = get_yearfrac(t0)
        t0_dt = dt.datetime.strptime(t0, '%Y-%m-%d')
        tmin_dt = dt.datetime.strptime(tmin, '%Y-%m-%d')
        tmax_dt = dt.datetime.strptime(tmax, '%Y-%m-%d')
        t_data = []
        s_data = []
        h_data = []
        for idx in ['ThMD', 'AHN1', 'AHN2', 'AHN3', 'AHN4']:
            if idx != 'ThMD':
                if -999 not in [EUP[f'{q}100_{idx}'] for q in 'hst'] and '' not in [EUP[f'{q}100_{idx}'] for q in 'hst']:
                    h_data.append(EUP[f'h100_{idx}'])
                    s_data.append(EUP[f's100_{idx}'])
                    t_data.append(get_yearfrac(EUP[f't100_{idx}']))
            else:
                if -999 not in [EUP[f'{q}_{idx}'] for q in 'hst'] and '' not in [EUP[f'{q}_{idx}'] for q in 'hst']:
                    h_data.append(EUP[f'h_{idx}'])
                    s_data.append(EUP[f's_{idx}'])
                    t_data.append(get_yearfrac(EUP[f't_{idx}']))
        InSAR_h = []
        InSAR_t = []
        InSAR_s = []
        for idx in [37, 88, 110, 161]:
            if f'h_{idx}' in EUP['InSAR'].keys():
                for i in range(np.array(EUP['InSAR'][f'h_{idx}']).shape[0]):
                    if not np.isnan(EUP['InSAR'][f'h_{idx}'][i]) and type(EUP['InSAR'][f't_{idx}'][i]) == str and not np.isnan(EUP['InSAR'][f's_{idx}'][i]):
                        InSAR_h.append(EUP['InSAR'][f'h_{idx}'][i])
                        InSAR_t.append(EUP['InSAR'][f't_{idx}'][i])
                        InSAR_s.append(EUP['InSAR'][f's_{idx}'][i])
        t_obs = []
        y_obs = []
        s_obs = []
        for i in range(len(h_data)):
            y_obs.append(h_data[i])
            t_obs.append(t_data[i])
            s_obs.append(s_data[i])
        for i in range(len(InSAR_t)):
            y_obs.append(InSAR_h[i])
            t_obs.append(get_yearfrac(InSAR_t[i]))
            s_obs.append(InSAR_s[i]) # 1 cm
        
        if t_data == []: # cannot estimate h0
            boundaries = [[0, 0.1], #xP
                          [0, 0.1], #xE
                          [-0.1, 0], #xI
                          [1, 365]]#, # tau
                          #[-10, 100]]# h0
            initial_params = initial_params[:-1]
        else:
            boundaries = [[0, 0.1], #xP
                          [0, 0.1], #xE
                          [-0.1, 0], #xI
                          [1, 365], # tau
                          [-10, 100]]# h0

        for i in range(len(initial_params)):
            if initial_params[i] is None:
                initial_params[i] = np.nan
            if initial_params[i] < boundaries[i][0] or np.isnan(initial_params[i]):
                initial_params[i] = (boundaries[i][0]+boundaries[i][1]) / 2
            if initial_params[i] > boundaries[i][1] or np.isnan(initial_params[i]):
                initial_params[i] = (boundaries[i][0]+boundaries[i][1]) / 2
        
            
        
        boundaries = np.array(boundaries).T
        
        #def get_model_estimates(params, InSAR_t, h_t, t0_dt, tmin_dt, tmax_dt, t0_yf, tmin_yf, tmax_yf, etmgeg):
        def model(t, *params, InSAR_t=InSAR_t, h_t=t_data, t0_dt=t0_dt, tmin_dt=tmin_dt, tmax_dt=tmax_dt, t0_yf=t0_yf, tmin_yf=tmin_yf, tmax_yf=tmax_yf,
                  etmgeg=etmgeg, return_xIfrac=False):#, tau=tau):
            xP = params[0]
            xE = params[1]
            xI = params[2]
            #tau = params[3]
            #h0 = params[3]
            tau = params[3]
            if len(h_t) > 0:
                h0 = params[4]
            else:
                h0 = 0
            
            InSAR_h = []
            h_h = []
            ndays = (tmax_dt-tmin_dt).days
            
            R_array = np.zeros((ndays, ))
            I_array = np.zeros((ndays, ))
            t_array = np.zeros((ndays, ))
            t0_idx = None
            
            for day in range(ndays):
                for backwards_day in range(int(tau)+1):
                    cur_day = tmin_dt + dt.timedelta(days=day-backwards_day)
                    try:
                        Pt = etmgeg[cur_day][0]
                        Et = etmgeg[cur_day][1]
                        if backwards_day == int(tau):
                            R_array[day] += (Pt*xP - Et*xE) * (tau % 1)
                        else:
                            R_array[day] += (Pt*xP - Et*xE)
                            
                    except KeyError:
                        continue
            
                if R_array[day] <= 0:
                    I_array[day] = I_array[day-1] + xI
                else:
                    I_array[day] = I_array[day-1]
                
                date = tmin_dt + dt.timedelta(days=day)
                t_array[day] = get_yearfrac('{:0>4d}-{:0>2d}-{:0>2d}'.format(date.year, date.month, date.day))
                if date == t0_dt:
                    t0_idx = day
            
            t_lst = list(t_array)
            if t0_idx is None:
                raise ValueError('t0 < tmax!')
            
            xIfrac = sum(1*R_array<=0)/R_array.shape[0]
            
            R_array -= R_array[t0_idx]
            I_array -= I_array[t0_idx]
            
            for t in InSAR_t:
                yft = get_yearfrac(t)
                tidx = t_lst.index(yft)
                InSAR_h.append(R_array[tidx]+I_array[tidx])
            
            for t in h_t:
                if t < tmin_yf:
                    elev = h0 + R_array[0] + I_array[0] + (dt.datetime.strptime(retrieve_epoch(t), '%Y-%m-%d') - tmin_dt).days * xI * xIfrac
                else:
                    tidx = t_lst.index(t)
                    elev = h0 + R_array[tidx]+I_array[tidx]
                h_h.append(elev)
    
            y_est = []        
            for i in range(len(h_h)):
                y_est.append(h_h[i])
            for i in range(len(InSAR_t)):
                y_est.append(InSAR_h[i])
            
            if return_xIfrac:
                return xIfrac
            
            return y_est
    
        if len(y_obs) <= 5:
            return no_insar_model()
        
        try:
            popt, pcov = curve_fit(model,
                                   xdata=t_obs,
                                   ydata=y_obs,
                                   p0=initial_params,
                                   sigma=s_obs,
                                   bounds=boundaries,
                                   method="trf",
                                   absolute_sigma=True,
                                   xtol=1e-6)
            
            stds = np.sqrt(np.diag(pcov))
            if len(t_data) > 0:
                xIfrac = model(t_obs, popt[0], popt[1], popt[2], popt[3], popt[4], return_xIfrac=True)
            else:
                xIfrac = model(t_obs, popt[0], popt[1], popt[2], popt[3], return_xIfrac=True)
                popt = np.append(popt, [-999])
                stds = np.append(stds, [-999])
                pcov_ = np.ones((5, 5)) * -999
                pcov_[:-1, :-1] = pcov[:, :]
                pcov = pcov_[:, :]
        except (ValueError, RuntimeError):  # Model does not converge!
            #print("Warning!! Optimal parameters not found, model did not converge!")
            #popt = [-999] * 5
            #stds = [-999] * 5
            #xIfrac = -999
            #pcov = np.ones((5, 5)) * -999
            return no_insar_model()
        
        
        return [popt, stds, max(t_obs) - min(t_obs), xIfrac, pcov]
        
    else: # no InSAR --> linear model 
        
        return no_insar_model()
    
    
def collect_all_models(EUPs, t0, tmin, tmax):
    all_models = {}
    for EUP in EUPs.keys():
        all_models[EUP] = fit_SPAMS_extended(EUPs[EUP], t0, tmin, tmax)
    return all_models


In [ ]:
def get_data(file):
    EUP_f = open(f'Data/EUPs/{file}.txt')
    raw = EUP_f.read()
    EUP_f.close()
    raw_EUP = raw.replace('nan', "'nan'").replace('array', 'np.array')
    EUP = eval(raw_EUP)

    if EUP['InSAR'] != -999:
        for keykey in [k for k in EUP['InSAR'].keys() if k[0] == 'h']:
            for d in range(len(EUP['InSAR'][keykey])):
                if EUP['InSAR'][keykey][d] == 'nan':
                    EUP['InSAR'][keykey][d] = np.nan

    
    y = []
    y_insar = []
    t = []
    t_insar = []
    s = []
    s_insar = []
    min_t = None
    max_t = None

    if EUP['EUP_type'] == 'brp':
        for key in ['100_AHN1', '100_AHN2', '100_AHN3', '100_AHN4', '_ThMD']:
            if EUP[f't{key}'] not in [-999, '']:
                y.append(EUP[f'h{key}'])
                t.append(EUP[f't{key}'])
                s.append(max(EUP[f's{key}'], 0.001))

        if EUP['InSAR'] != -999:
            keys = [key[1:] for key in EUP['InSAR'].keys() if key[0] == 't']
            for key in keys:
                for t_ in range(len(EUP['InSAR'][f't{key}'])):
                    if not np.isnan(EUP['InSAR'][f'h{key}'][t_]):
                        t_insar.append(EUP['InSAR'][f't{key}'][t_])
                        y_insar.append(EUP['InSAR'][f'h{key}'][t_])
                        s_insar.append(0.01)

    else:
        for key in ['100_AHN1', '100_AHN2', '100_AHN3', '100_AHN4', '_ThMD']:
            if EUP[f't{key}'] not in [-999, '']:
                y.append(EUP[f'h{key}'])
                t.append(EUP[f't{key}'])
                s.append(EUP[f's{key}'])
    full_y = []
    full_t = []
    full_s = []
    for d in range(len(y)):
        full_y.append(y[d])
        full_t.append(get_yearfrac(t[d]))
        full_s.append(s[d])
    for d in range(len(y_insar)):
        full_y.append(y_insar[d]+EUP['h0'])
        full_t.append(get_yearfrac(t_insar[d]))
        full_s.append(s_insar[d])
    return full_y, full_t, full_s

def plot_model(file):
    EUP_f = open(f'Data/EUPs/{file}.txt')
    raw = EUP_f.read()
    EUP_f.close()
    raw_EUP = raw.replace('nan', "'nan'").replace('array', 'np.array')
    EUP = eval(raw_EUP)

    if EUP['InSAR'] != -999:
        for keykey in [k for k in EUP['InSAR'].keys() if k[0] == 'h']:
            for d in range(len(EUP['InSAR'][keykey])):
                if EUP['InSAR'][keykey][d] == 'nan':
                    EUP['InSAR'][keykey][d] = np.nan
    tt = []
    h = []
    cur_t = t_eval_min
    while get_yearfrac('{:0>4d}-{:0>2d}-{:0>2d}'.format(cur_t.year, cur_t.month, cur_t.day)) < tmax_yf:
        tt.append('{:0>4d}-{:0>2d}-{:0>2d}'.format(cur_t.year, cur_t.month, cur_t.day))
        cur_t += dt.timedelta(days=1)

    xI = EUP['xI']
    xP = EUP['xP']
    xE = EUP['xE']
    h0 = EUP['h0']
    tau = EUP['tau']

    ndays = (tmax_dt-tmin_dt).days

    R_array = np.zeros((ndays, ))
    I_array = np.zeros((ndays, ))
    t_array = np.zeros((ndays, ))
    t0_idx = None

    for day in range(ndays):
        for backwards_day in range(int(tau)+1):
            cur_day = tmin_dt + dt.timedelta(days=day-backwards_day)
            try:
                Pt = etmgeg[cur_day][0]
                Et = etmgeg[cur_day][1]
                if backwards_day == int(tau):
                    R_array[day] += (Pt*xP - Et*xE) * (tau % 1)
                else:
                    R_array[day] += (Pt*xP - Et*xE)

            except KeyError:
                continue

        if R_array[day] <= 0:
            I_array[day] = I_array[day-1] + xI
        else:
            I_array[day] = I_array[day-1]

        date = tmin_dt + dt.timedelta(days=day)
        t_array[day] = get_yearfrac('{:0>4d}-{:0>2d}-{:0>2d}'.format(date.year, date.month, date.day))
        if date == t0_dt:
            t0_idx = day

    t_lst = list(t_array)
    if t0_idx is None:
        raise ValueError('t0 > tmax!')

    xIfrac = sum(1*R_array<=0)/R_array.shape[0]

    R_array -= R_array[t0_idx]
    I_array -= I_array[t0_idx]


    for t in tt:
        if get_yearfrac(t) < tmin_yf:
            elev = h0 + R_array[0] + I_array[0] + (dt.datetime.strptime(t, '%Y-%m-%d') - tmin_dt).days * xI * xIfrac
        else:
            tidx = t_lst.index(get_yearfrac(t))
            elev = h0 + R_array[tidx]+I_array[tidx]
        h.append(elev)

    return [get_yearfrac(t) for t in tt], h

def convert_value_to_hex(value, vmin=-10, vmax=10):
    pct = min(max((value - vmin) / (vmax - vmin), 0), 1)
    clr = (int(255 * min(1, 2 - 2 * pct)), 
           int(255 * (1 - 2 * abs(0.5 - pct))), 
           int(255 * min(1, 2 * pct)))
    #clr = (int(255 * max(0, (1 - 2 * pct))), 0, (int(255 * max(2*pct - 1, 0))))
    hx = '#'
    for c in clr:
        hx += "{:0>2s}".format(hex(c)[2:])
    return hx

def map_index(EUP):
    """
    This function maps the index of the gpkg EUP to the txt EUP, as these are not the same:
    the gpkg maintains the original numbering before ~50000 EUPs were removed, the txt EUP
    starts counting from 0 without gaps
    :param EUP: the gpkg index of the EUP
    :return: the txt index of the EUP
    """
    keys = list(gpkg_data.keys())
    print(keys.index(EUP))
    return keys.index(EUP)

def map_vis_index(EUP):
    """
    This function maps the index of the gpkg EUP to the visualisation EUP, as these are not the same:
    the gpkg maintains the original numbering before ~50000 EUPs were removed, the visualisation EUP
    starts counting from 0 without gaps and has EUPs with no data removed
    :param EUP: the gpkg index of the EUP
    :return: the vis index of the EUP
    """ 
    vis_id = None
    for att_id in vis_data.keys():
        if gpkg_data[EUP]['crd_dec'] == vis_data[att_id]['crd_dec']:
            vis_id = att_id
            break
    return vis_id

def convert_gpkg_to_wgs84(gpkg):
    for EUP in gpkg.keys():
        gpkg[EUP]['crd_wgs'] = convert_rd_to_wgs84(gpkg_data[EUP]['crd_dec'][0])
    return gpkg


In [ ]:
gpkg_data = convert_gpkg_to_wgs84(gpkg_data)

In [ ]:
output = widgets.Output()

In [ ]:
def plot_overview(plot='xI'):
    with output:
        clear_output(wait=True)
        display(main_plot)
    if plot != 'Select':
        vdir = {'xI': [-10, 10],
                's_xI': [0, 10],
                'h0': [-8, 2],
                's_h0': [0, 10]}
                    
        with output:
            CarDB = basemap_to_tiles(basemaps.CartoDB.Positron)
            CarDB.base = True
            CarDB.name = 'Map (CartoDB)'
            
            m = Map(center=list(np.mean(np.array(aoi_wgs_poly), axis=0)), zoom=12, min_zoom=12, close_popup_on_click=False , layers = [CarDB])

            AoI = LayerGroup(name = 'AoI')
            eup_groups = []
            eup_groups.append(LayerGroup())           

            line = Polyline(locations = aoi_wgs_poly,
                        weight = 2,
                        color= '#000000', linestyle='--', fill=False) 

            highlighter = Polyline(locations = [[0, 0], [0, 0]],
                                   weight=3,
                                   color='#5abf4d', fill=False, name='Selection')
            
            AoI.add_layer(line)

            def update_data(b):
                with out4:
                    print(f'Should remove {b.description.split(" ")[-1]}')
            
            def plot_data(EUP, m):
                def button_click(**kwargs):
                    clear_output(wait=True)
                    txt_eup = map_index(EUP)
                    vis_eup = map_vis_index(EUP)                        
                    t, h = plot_model(txt_eup)
                    y, yt, ys = get_data(txt_eup)
                    EUP_f = open(f'Data/EUPs/{txt_eup}.txt')
                    raw = EUP_f.read()
                    EUP_f.close()
                    raw_EUP = raw.replace('nan', "'nan'").replace('array', 'np.array')
                    data = eval(raw_EUP)
                    m.remove_layer(m.layers[-1])
                    highlighter = Polyline(locations = convert_rd_to_wgs84(gpkg_data[EUP]['crd_dec'][0]),
                                weight=3,
                                color='#5abf4d', fill=False, name='Selection')
                    m.add_layer(highlighter)
                    with out1:
                        clear_output(wait=True)
                        plt.figure(figsize=(11,5))
                        plt.plot(t, h, label='Model fit')
                        plt.errorbar(yt, y, yerr=ys, fmt='o', ecolor='r', label='Observations')
                        plt.grid(True)
                        plt.xlabel('Time [y]')
                        plt.ylabel('Elevation [m-NAP1997]')
                        plt.legend()
                        plt.show()
                    with out3:
                        clear_output(wait=True)
                        if vis_eup is None:
                            print('This spatial unit was filtered out of the resulting dataset due to a failed model estimate')
                        else:
                            if data['model_mode'] == "SPAMS":
                                print(f'''GPKG spatial unit {EUP} (TXT {txt_eup}, VIS {vis_eup})
    ----------
    Estimated parameters ({data['model_mode'] + ('-extended' if data['model_mode'] == 'SPAMS' else '')} model)
    
    Elevation (1 Jan 2015): {round(vis_data[vis_eup]['h2015_m'], 3)}±{round(vis_data[vis_eup]['sigma_h2015_m'], 3)} m - NAP
    XI (irreversible): {round(vis_data[vis_eup]['xI_mpy']*1000, 1)}±{round(vis_data[vis_eup]['sigma_xI_mpy']*1000, 1)} mm/y ({round(vis_data[vis_eup]['xI_frac_1000pct']*1000, 1)}% active)
    XP (precipitation): {round(vis_data[vis_eup]['xP_mpmm']*1000, 3)}±{round(vis_data[vis_eup]['sigma_xP_mpmm']*1000, 3)} mm/mm
    XE (evapotranspiration): {round(vis_data[vis_eup]['xE_mpmm']*1000, 3)}±{round(vis_data[vis_eup]['sigma_xE_mpmm']*1000, 3)} mm/mm
    Tau (delay): {round(vis_data[vis_eup]['tau_kd']*1000, 1)}±{round(vis_data[vis_eup]['sigma_tau_kd']*1000, 1)} days
    
    ----------
    Derived variables
    
    Mean Yearly Irreversible Subsidence: {round(vis_data[vis_eup]['mean_yearly_irreversible_subsidence_MYIS_mpy']*1000, 1)}±{round(vis_data[vis_eup]['sigma_MYIS_mpy']*1000, 1)} mm/y 
    Observation timespan: {round(vis_data[vis_eup]['timespan_ky']*1000, 1)} y
    # observations: {int(1000*vis_data[vis_eup]['n_observations_1000'])}
    Cumulative irreversible subsidence since 2000: {round(vis_data[vis_eup]['cum_irreversible_subsidence_since_2000_m'] * 1000, 1)} mm
    
    ----------
    Covariances
    cov(h0, xI): {round(vis_data[vis_eup]['rho_h2015_xI_1000']*1000 * vis_data[vis_eup]['sigma_h2015_m'] * vis_data[vis_eup]['sigma_xI_mpy']*1000, 3)}
    cov(h0, xP): {round(vis_data[vis_eup]['rho_h2015_xP_1000']*1000 * vis_data[vis_eup]['sigma_h2015_m'] * vis_data[vis_eup]['sigma_xP_mpmm']*1000, 3)}
    cov(h0, xE): {round(vis_data[vis_eup]['rho_h2015_xE_1000']*1000 * vis_data[vis_eup]['sigma_h2015_m'] * vis_data[vis_eup]['sigma_xE_mpmm']*1000, 3)}
    cov(h0, tau): {round(vis_data[vis_eup]['rho_h2015_tau_1000']*1000 * vis_data[vis_eup]['sigma_h2015_m'] * vis_data[vis_eup]['sigma_tau_kd']*1000, 3)}
    cov(xI, xE): {round(vis_data[vis_eup]['rho_xI_xE_1000']*1000 * vis_data[vis_eup]['sigma_xI_mpy']*1000 * vis_data[vis_eup]['sigma_xE_mpmm']*1000, 3)}
    cov(xI, xP): {round(vis_data[vis_eup]['rho_xI_xP_1000']*1000 * vis_data[vis_eup]['sigma_xI_mpy']*1000 * vis_data[vis_eup]['sigma_xP_mpmm']*1000, 3)}
    cov(xI, tau): {round(vis_data[vis_eup]['rho_xI_tau_1000']*1000 * vis_data[vis_eup]['sigma_xI_mpy']*1000 * vis_data[vis_eup]['sigma_tau_kd']*1000, 3)}
    cov(xP, xE): {round(vis_data[vis_eup]['rho_xP_xE_1000']*1000 * vis_data[vis_eup]['sigma_xP_mpmm']*1000 * vis_data[vis_eup]['sigma_xE_mpmm']*1000, 3)}
    cov(xP, tau): {round(vis_data[vis_eup]['rho_xP_tau_1000']*1000 * vis_data[vis_eup]['sigma_xP_mpmm']*1000 * vis_data[vis_eup]['sigma_tau_kd']*1000, 3)}
    cov(xE, tau): {round(vis_data[vis_eup]['rho_xE_tau_1000']*1000 * vis_data[vis_eup]['sigma_xE_mpmm']*1000 * vis_data[vis_eup]['sigma_tau_kd']*1000, 3)}
    
    ---------
    Statistics
    
    Significant irreversible subsidence detected: {vis_data[vis_eup]['significant_irreversible_subsidence_detected_2s_1000']*1000} (2 sigma) / {vis_data[vis_eup]['significant_irreversible_subsidence_detected_3s_1000']*1000} (3 sigma)
    OMT sustained: {vis_data[vis_eup]['OMT_sust_2s_1000']*1000} (2 sigma) / {vis_data[vis_eup]['OMT_sust_3s_1000']*1000} (3 sigma)
    Discriminatory power: {round(vis_data[vis_eup]['discriminatory_power_2s_1000pct']*1000, 2)}% (2 sigma) / {round(vis_data[vis_eup]['discriminatory_power_3s_1000pct']*1000, 2)}% (3 sigma)
    Minimum Detectable Displacement (80% confidence): {round(vis_data[vis_eup]['MDD_80p_mpy']*1000, 2)} mm/y
    
    ''')
                            else: 
                                print(f'''GPKG spatial unit {EUP} (TXT {txt_eup}, VIS {vis_eup})
----------
Estimated parameters ({data['model_mode'] + ('-extended' if data['model_mode'] == 'SPAMS' else '')} model)

Elevation (1 Jan 2015): {round(vis_data[vis_eup]['h2015_m'], 3)}±{round(vis_data[vis_eup]['sigma_h2015_m'], 3)} m - NAP
XI (irreversible): {round(vis_data[vis_eup]['xI_mpy']*1000, 1)}±{round(vis_data[vis_eup]['sigma_xI_mpy']*1000, 1)} mm/y ({round(vis_data[vis_eup]['xI_frac_1000pct']*1000, 1)}% active)

----------
Derived variables

Mean Yearly Irreversible Subsidence: {round(vis_data[vis_eup]['mean_yearly_irreversible_subsidence_MYIS_mpy']*1000, 1)}±{round(vis_data[vis_eup]['sigma_MYIS_mpy']*1000, 1)} mm/y 
Observation timespan: {round(vis_data[vis_eup]['timespan_ky']*1000, 1)} y
# observations: {int(1000*vis_data[vis_eup]['n_observations_1000'])}
Cumulative irreversible subsidence since 2000: {round(vis_data[vis_eup]['cum_irreversible_subsidence_since_2000_m'] * 1000, 1)} mm

----------
Covariances
cov(h0, xI): {round(vis_data[vis_eup]['rho_h2015_xI_1000']*1000 * vis_data[vis_eup]['sigma_h2015_m'] * vis_data[vis_eup]['sigma_xI_mpy']*1000, 3)}

---------
Statistics

Significant irreversible subsidence detected: {vis_data[vis_eup]['significant_irreversible_subsidence_detected_2s_1000']*1000} (2 sigma) / {vis_data[vis_eup]['significant_irreversible_subsidence_detected_3s_1000']*1000} (3 sigma)
OMT sustained: {vis_data[vis_eup]['OMT_sust_2s_1000']*1000} (2 sigma) / {vis_data[vis_eup]['OMT_sust_3s_1000']*1000} (3 sigma)
Discriminatory power: {round(vis_data[vis_eup]['discriminatory_power_2s_1000pct']*1000, 2)}% (2 sigma) / {round(vis_data[vis_eup]['discriminatory_power_3s_1000pct']*1000, 2)}% (3 sigma)
Minimum Detectable Displacement (80% confidence): {round(vis_data[vis_eup]['MDD_80p_mpy']*1000, 2)} mm/y

''')
                        #print(f'GPKG EUP {EUP} (TXT {txt_eup}, VIS {vis_eup})')
                        #print(f"Mean Yearly Irreversible Subsidence: {round(data['xI'] * data['xI_frac'] * 365.2425 * 1000, 1)}±{round(data['s_xI'] * data['xI_frac'] * 365.2425 * 1000, 1)} mm/y")
                        #print(f"Elevation (1 Jan 2015): {round(data['h0'], 3)}±{round(data['s_h0'], 3)} m-NAP1997")
                        
                    #with out3:
                    #    clear_output(wait=True)
                        #if data['t_ThMD'] not in ['', -999]:
                        #    thmd_button = widgets.Button(description='Remove ThMD')
                        #else:
                        #    thmd_button = widgets.Button(description='Remove ThMD', disabled=True)
                        #thmd_button.on_click(update_data)
                        #if data['t80_AHN1'] not in ['', -999]:
                        #    ahn1_button = widgets.Button(description='Remove AHN1')
                        #else:
                        #    ahn1_button = widgets.Button(description='Remove AHN1', disabled=True)
                        #ahn1_button.on_click(update_data)
                        #if data['t80_AHN2'] not in ['', -999]:
                        #    ahn2_button = widgets.Button(description='Remove AHN2')
                        #else:
                        #    ahn2_button = widgets.Button(description='Remove AHN2', disabled=True)
                        #ahn2_button.on_click(update_data)
                        #if data['t80_AHN3'] not in ['', -999]:
                        #    ahn3_button = widgets.Button(description='Remove AHN3')
                        #else:
                        #    ahn3_button = widgets.Button(description='Remove AHN3', disabled=True)
                        #ahn3_button.on_click(update_data)
                        #if data['t80_AHN4'] not in ['', -999]:
                        #    ahn4_button = widgets.Button(description='Remove AHN4')
                        #else:
                        #    ahn4_button = widgets.Button(description='Remove AHN4', disabled=True)
                        #ahn4_button.on_click(update_data)
                        #if data['InSAR'] not in ['', -999]:
                        #    insar_button = widgets.Button(description='Remove InSAR')
                        #else:
                        #    insar_button = widgets.Button(description='Remove InSAR', disabled=True)
                        #insar_button.on_click(update_data)
                        #eup_button = widgets.Button(description='Remove EUP')
                        #display(eup_button)
                        #display(thmd_button)
                        #display(ahn1_button)
                        #display(ahn2_button)
                        #display(ahn3_button)
                        #display(ahn4_button)
                        #display(insar_button)
                    #with out4:
                    #    print(f'Analysing GPKG EUP {EUP} (TXT {txt_eup})')
                    with out2:
                        print(f'''GPKG spatial unit {EUP} (TXT {txt_eup}, VIS {vis_eup})''')
                    with out4:
                        print(f'''GPKG spatial unit {EUP} (TXT {txt_eup}, VIS {vis_eup})''') 
                        
                return button_click
            out1 = widgets.Output()
            out2 = widgets.Output()
            out3 = widgets.Output()
            out4 = widgets.Output()
        
            tab = widgets.Tab(children = [out1, out2])
            tab.set_title(0, 'Estimated Model')
            tab.set_title(1, 'Unused')

            tab2 = widgets.Tab(children = [out3, out4])
            tab2.set_title(0, 'Parameters')
            tab2.set_title(1, 'Unused')
            
            m.add(FullScreenControl())
            m.add_layer(highlighter)

            display(tab2)
            display(tab)
            display(m)
            ct = 0
            with out1:
                clear_output(wait=True)
                print('Loading spatial units...')
                print(f'{ct}/{len(gpkg_data.keys())}')
            for EUP in list(gpkg_data.keys()):
                try:
                    if plot in ['xI', 's_xI']:
                        val = gpkg_data[EUP][plot] * gpkg_data[EUP]['xI_frac'] * 365.2425 * 1000 # mm/y
                    elif plot in ['h0']:
                        val = gpkg_data[EUP][plot] # m
                    elif plot in ['s_h0']:
                        val = gpkg_data[EUP][plot] * 100 # cm
                    else:
                        val = gpkg_data[EUP][plot] * 1000
                    polygon = Polygon(
                        locations = gpkg_data[EUP]['crd_wgs'],
                        weight=0,
                        fill_color=convert_value_to_hex(val, vdir[plot][0], vdir[plot][1]),
                        fill_opacity=1)
                    polygon.on_click(plot_data(EUP, m=m))
                    eup_groups[-1].add_layer(polygon)
                    ct += 1
                    if ct % MAX_GROUP_SIZE == 0:
                        with out1:
                            clear_output(wait=True)
                            print('Loading spatial units...')
                            print(f'{ct}/{len(gpkg_data.keys())}')
                        m.add_layer(eup_groups[-1])
                        eup_groups.append(LayerGroup())
                except TypeError:
                    pass
            if ct % MAX_GROUP_SIZE != 0:      
                m.add_layer(eup_groups[-1])
                with out1:
                    clear_output(wait=True)
                    print('Loaded spatial units!')
                    print(f'Click to analyse a spatial unit')
                with out2:
                    clear_output(wait=True)
                    print('Loaded spatial units!')
                    print(f'Click to analyse a spatial unit')
                with out3:
                    clear_output(wait=True)
                    print('Loaded spatial units!')
                    print(f'Click to analyse a spatial unit')
                with out4:
                    clear_output(wait=True)
                    print('Loaded spatial units!')
                    print(f'Click to analyse a spatial unit')
            m.add_layer(AoI)

In [ ]:
plot_drop = widgets.Dropdown(
        options=['Select', 'xI', 's_xI', 'h0', 's_h0'],
        value='Select',
        description='Parameter:',
        disabled=False,
    )
main_plot = interactive(plot_overview, plot=plot_drop)


In [ ]:
%matplotlib ipympl
button = widgets.Button(description="Run Me!")


display(button, output)

def on_button_clicked(b):
    with output:
        clear_output()
        display(main_plot)
        
button.on_click(on_button_clicked)